In [ ]:
import arviz as az
import yaml
from tbh.paths import OUTPUT_PARENT_FOLDER, DATA_FOLDER
from tbh import runner_tools as rt
from tbh.model import get_tb_model
from estival.model import BayesianCompartmentalModel

from matplotlib import pyplot as plt 
plt.style.use("ggplot")

In [ ]:
analysis_path = OUTPUT_PARENT_FOLDER / "46127547_full_analysis_3scenarios" / "task_1"
idata = az.from_netcdf(analysis_path / "idata.nc")

with open(analysis_path / "timings.yaml" , "r") as f:
    docs = list(yaml.safe_load_all(f))

model_config = docs[1]
analysis_config = docs[2]

In [ ]:
params, priors, tv_params = rt.get_parameters_and_priors()

model = get_tb_model(model_config, tv_params)
bcm = BayesianCompartmentalModel(model, params, priors, rt.targets)

bcm_dict = {"baseline": bcm}
for sc in analysis_config['scenarios']:
    assert sc != "baseline", "Please use scenario name different from 'baseline'"
    sc_params, _, sc_tv_params = rt.get_parameters_and_priors(DATA_FOLDER / f"{sc}.xlsx")
    sc_params, sc_tv_params = params | sc_params, tv_params | sc_tv_params
    sc_model = get_tb_model(model_config, sc_tv_params)
    sc_bcm = BayesianCompartmentalModel(sc_model, sc_params, priors, rt.targets)
    bcm_dict[sc] = sc_bcm

In [ ]:
full_runs, unc_dfs = rt.run_full_runs(
    bcm_dict, idata, analysis_config['burn_in'], 100
)

In [ ]:
full_runs['scenario_1'].results.columns.get_level_values(0).unique()

In [ ]:
for sc in full_runs:
    df = full_runs[sc].results
    df.to_csv(analysis_path / f"outputs_{sc}.csv")